# Import

In [1]:
import json
from datetime import datetime, timedelta

# Data Reader/Writter Functions
```python
#Read json
load("file_path.txt")
#Convert str to json
str_to_json(data)
#Write json to txt
write(data, "file_path.txt")
```

In [58]:
def load(filepath):
    with open(filepath, 'r', encoding='cp037') as f:
        return json.load(f)

def write(data, filepath):
    json_str = json.dumps(data, ensure_ascii=False, indent=2)
    with open(filepath, 'w', encoding='cp037') as f:
        f.write(json_str)

def str_to_json(s):
    return eval(s)

# Data Reader

In [3]:
class datasets:
    def __init__(self):
        self.users = load('datasets/users.txt')
        self.menu = load('datasets/menu.txt')
        self.ingredients = load('datasets/ingredients.txt')
        self.ingredient_order = load('datasets/ingredient_order.txt')
        self.orders = load('datasets/orders.txt')
        self.finance = load('datasets/finance.txt')
        self.feedbacks = load('datasets/feedbacks.txt')

    def get_users(self):
        return self.users

    def get_menu(self):
        return self.menu

    def get_iventory(self):
        return self.ingredients

    def get_ingredient_order(self):
        return self.ingredient_order

    def get_orders(self):
        return self.orders

    def get_finance(self):
        return self.finance

    def get_feedback(self):
        return self.feedbacks

# Linear Regression

ŷ / y-hat stands for prediction

Linear Equation: ŷ = ax + b

Loss (MSE Loss) = (y - ŷ)^2

Expansion of Loss: L = y^2 - 2y(ax + b) + (ax + b)^2

                = y^2 - 2ayx - 2yb + a^2x^2 + 2axb + b^2
Derivative of a: dL/da = -2yx + 2ax^2 + 2xb

Derivative of b: dL/db = -2y + 2ax + 2b

New Loss(a) = a - learning rate(lr) * da

New Loss(b) = a - learning rate(lr) * db

In [4]:
class Model:
    def __init__(self):
        with open('augmented_orders.json', 'r', encoding='utf-8') as f:
            self.orders_data = json.load(f)

        self.ingredients_data = load('datasets/ingredients.txt')

        completed_orders = [o for o in self.orders_data['orders'] if o['status'] == 'Completed']
        order_time_map = {o['order_id']: o['order_time'] for o in completed_orders}

        daily_dish_sales = {}
        for item in self.orders_data['order_items']:
            oid = item['order_id']
            if oid not in order_time_map:
                continue
            date_str = order_time_map[oid][:10]
            dish_id = item['dish_id']
            qty = item['quantity']
            key = (date_str, dish_id)
            daily_dish_sales[key] = daily_dish_sales.get(key, 0) + qty

        recipe_dict = {}
        for r in self.ingredients_data['recipes']:
            recipe_dict[r['dish_id']] = [(ing['ingredient_id'], ing['quantity']) for ing in r['ingredients']]

        daily_ingredient_sales = {}
        for (date_str, dish_id), dish_qty in daily_dish_sales.items():
            if dish_id not in recipe_dict:
                continue
            for ing_id, ing_qty_per_dish in recipe_dict[dish_id]:
                key = (date_str, ing_id)
                daily_ingredient_sales[key] = daily_ingredient_sales.get(key, 0) + dish_qty * ing_qty_per_dish

        dates = sorted(list(set(date for date, _ in daily_ingredient_sales.keys())))
        date_to_num = {date: i for i, date in enumerate(dates)}
        ingredient_ids = list(set(ing_id for _, ing_id in daily_ingredient_sales.keys()))

        ingredient_time_series = {ing_id: [0.0] * len(dates) for ing_id in ingredient_ids}
        for (date, ing_id), qty in daily_ingredient_sales.items():
            idx = date_to_num[date]
            ingredient_time_series[ing_id][idx] = qty

        self.weekly_results = {}
        for ing_id in ingredient_ids:
            y = ingredient_time_series[ing_id]
            x = list(range(len(y)))
            a, b = self.gradient_descent_linear_regression(x, y)
            total_pred_qty = sum(max(a * (len(dates) - 1 + i) + b, 0) for i in range(1, 8))
            self.weekly_results[ing_id] = total_pred_qty

    def gradient_descent_linear_regression(self, x_list, y_list, lr=0.01, epochs=5000):
        a, b = 0.0, 0.0
        n = len(x_list)
        if n == 0:
            return 0.0, 0.0
        for _ in range(epochs):
            da = sum((a * x + b - y) * x for x, y in zip(x_list, y_list)) * 2 / n
            db = sum((a * x + b - y) for x, y in zip(x_list, y_list)) * 2 / n
            a -= lr * da
            b -= lr * db
        return a, b

    def print_output(self):
        name_map = {i['ingredient_id']: i['name'] for i in self.ingredients_data['ingredients']}
        unit_map = {i['ingredient_id']: i.get('unit', 'unit') for i in self.ingredients_data['ingredients']}
        print("Prediction of total demand for each ingredient in the next week (7 days):")
        for ing_id, qty in sorted(self.weekly_results.items(), key=lambda x: -x[1]):
            print(f"- {name_map.get(ing_id, ing_id)}: {qty:.2f} {unit_map.get(ing_id)}")

# Main system
```python
#Example of data getter
system = system_manager()

for SubClass in (Manager, Cashier, Chef, Customer):
    inst = SubClass()
    decrypted_data = inst.retrieve_data(system)
    decrypted_data = str_to_json(decrypted_data)
    print(f"{SubClass.__name__} decrypted:", decrypted_data)
```

In [5]:
if isinstance(__builtins__, dict):
    _B_CHR = __builtins__['chr']
    _B_ORD = __builtins__['ord']
else:
    _B_CHR = __builtins__.chr
    _B_ORD = __builtins__.ord

In [59]:
class system_manager:
    def __init__(self):
        data_banker = datasets()
        self.users_available = data_banker.get_users()

        """Encryption System"""
        #Different roles are distributted with different datasets
        self._plain_data = {
            Manager: {"user": data_banker.get_users(),
                      "order": data_banker.get_orders(),
                      "inventory": data_banker.get_iventory(),
                      "ingredient_order": data_banker.get_ingredient_order(),
                      "finance": data_banker.get_finance(),
                      "feedbacks": data_banker.get_feedback()},
            Cashier: {"menu": data_banker.get_menu(),
                      "order": data_banker.get_orders()},
            Chef: {"ingr": data_banker.get_iventory()},
            Customer: {"menu": data_banker.get_menu(),
                      "feedback": data_banker.get_feedback()}
        }
        
        # Key for decryption
        self.caesar_shift = 3
        self.xor_key = 42
        self.mul_key = 2

    #According to the requester type: take out the corresponding plaintext and encrypt it before returning
    def get_data(self, requester) -> str:
        cls = type(requester)
        if cls not in self._plain_data:
            raise ValueError(f"Unknown requester: {cls}")
        plain = str(self._plain_data[cls])

        if cls is Manager:
            dt = self._encrypt_atbash(plain)
            return dt
        elif cls is Cashier:
            dt = self._encrypt_caesar(plain)
            return dt
        elif cls is Chef:
            dt = self._encrypt_xor(plain)
            return dt
        elif cls is Customer:
            dt = self._encrypt_mul(plain)
            return dt

    # ——Four encryption method——
    def _encrypt_atbash(self, data: str) -> str:
        result = []
        for c in data:
            if 'a' <= c <= 'z':
                result.append(_B_CHR(_B_ORD('z') - (_B_ORD(c) - _B_ORD('a'))))
            elif 'A' <= c <= 'Z':
                result.append(_B_CHR(_B_ORD('Z') - (_B_ORD(c) - _B_ORD('A'))))
            elif '0' <= c <= '9':
                result.append(_B_CHR(_B_ORD('9') - (_B_ORD(c) - _B_ORD('0'))))
            else:
                result.append(c)
        return ''.join(result)

    def _encrypt_caesar(self, data: str) -> str:
        shift = self.caesar_shift
        result = []
        for c in data:
            if 'a' <= c <= 'z' or 'A' <= c <= 'Z':
                base = 'a' if c.islower() else 'A'
                result.append(_B_CHR(
                    (_B_ORD(c) - _B_ORD(base) + shift) % 26
                    + _B_ORD(base)
                ))
            else:
                result.append(c)
        return ''.join(result)

    def _encrypt_xor(self, data: str) -> str:
        key = self.xor_key
        return "".join(f"{(_B_ORD(c) ^ key):02x}" for c in data)

    def _encrypt_mul(self, data: str) -> str:
        key = self.mul_key
        return ",".join(str(_B_ORD(c) * key) for c in data)

    """Login function"""
    def login(self, accID, password):
        for user in self.users_available['users']:
            if user["accID"] == accID and user["pass"] == password:
                return True, user['role']
        return False, None
        
    """Manager Permissions"""
    def manager_only(method):
        def wrapper(self, requester, *args, **kwargs):
            if requester.__class__.__name__ == 'Manager':
                return method(self, requester, *args, **kwargs)
            print(f"Permission denied: {method.__name__} requires Manager role.")
        return wrapper

    @manager_only
    def add_user(self, user):
        self.users_available['users'].append(user)
        write(self.users_available, "datasets/users.txt")

    @manager_only
    def write_users(self, requester, users_dict):
        write(users_dict, 'datasets/users.txt')

    @manager_only
    def write_orders(self, requester, orders_dict):
        write(orders_dict, 'datasets/orders.txt')

    @manager_only
    def write_ingredients(self, requester, inv_dict):
        write(inv_dict, 'datasets/ingredient_order.txt')

# Manager

In [60]:
class Manager:
    def __init__(self, system, panel):
        self.system = system
        self.panel = panel
        
    """Atbash Decryption Function"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted)

    def decrypt(self, data: str) -> str:
        result = []
        for c in data:
            if 'a' <= c <= 'z':
                result.append(_B_CHR(_B_ORD('z') - (_B_ORD(c) - _B_ORD('a'))))
            elif 'A' <= c <= 'Z':
                result.append(_B_CHR(_B_ORD('Z') - (_B_ORD(c) - _B_ORD('A'))))
            elif '0' <= c <= '9':
                result.append(_B_CHR(_B_ORD('9') - (_B_ORD(c) - _B_ORD('0'))))
            else:
                result.append(c)
        return ''.join(result)

    def get_item(self):
        system = system_manager()
        decrypted_data = self.retrieve_data(system)
        decrypted_data = str_to_json(decrypted_data)
        return decrypted_data

    """System Administration"""
    def approve(self, accID):
        for user in self.panel.pending_users:
            if user['accID'] == accID:
                self.panel.pending_users.remove(user)
                self.system.add_user(user)
                print(f"The administrator approved the registration of user: {accID}.")
                return
        print(f"No pending user named {accID} found.")

    def reject(self, accID):
        for user in self.panel.pending_users:
            if user['accID'] == accID:
                self.panel.pending_users.remove(user)
                print(f"The administrator denied registration for user {accID}.")
                return
        print(f"No pending user named {accID} found.")

    def list_approved(self):
        f_data = self.get_item()
        users_available = f_data['user']
        users = users_available.get('users', [])
        print("List of approved users:")
        for user in users:
            print(f"- {user['accID']} (Role: {user['role']})")
        return users

    def modify_user(self, accID, new_role=None, new_password=None):
        f_data = self.get_item()
        users_available = f_data['user']
        for user in users_available['users']:
            if user['accID'] == accID:
                if new_role:
                    user['role'] = new_role
                if new_password:
                    user['pass'] = new_password
                self.system.write_users(self, users_available)
                print(f"User updated: {accID}")
                return
        print(f"User {accID} not found")

    """Order Management"""
    def list_pending_orders(self):
        f_data = self.get_item()
        orders_data = f_data['order']
        pending = [o for o in orders_data['orders'] if o['status'].lower() != 'completed']
        if not pending:
            print("There are currently no outstanding orders.")
        else:
            print("List of uncompleted orders:")
            for o in pending:
                print(f"- OrderID: {o['order_id']}, CustomerID: {o['customer_id']}, Time: {o['order_time']}, Status: {o['status']}, Amount: {o['total']}")
        return pending

    def update_order_status(self, order_id, new_status):
        f_data = self.get_item()
        orders_data = f_data['order']
        for o in orders_data['orders']:
            if o['order_id'] == order_id:
                o['status'] = new_status
                self.system.write_orders(self, orders_data)
                print(f"Updated order {order_id} status to {new_status}")
                return
        print(f"Order {order_id} not found")

    """Financial Management"""
    def financial_summary_past_week(self):
        f_data = self.get_item()
        orders_data = f_data['order']
        all_orders = orders_data.get('orders', [])

        now = datetime.now().date()
        week_ago = now - timedelta(days=7)

        recent_orders = []
        for order in all_orders:
            try:
                order_time = datetime.fromisoformat(order['order_time']).date()
            except Exception as e:
                print(f"Order {order['order_id']} time format error: {e}")
                continue

            if week_ago <= order_time <= now and order['status'] == 'Completed':
                recent_orders.append(order)

        if not recent_orders:
            print("No orders in the past week.")
            return []

        print(f"Order statistics in the past week (a total of {len(recent_orders)} orders):")
        total_amount = 0.0
        for order in recent_orders:
            print(f"- OrderID: {order['order_id']}, CustomerID: {order['customer_id']}, Time: {order['order_time']}, Status: {order['status']}, Amount: {order['total']}")
            total_amount += order.get('total', 0.0)

        print(f"Total amount：{total_amount:.2f}")
        return recent_orders

    def prediction_output(self):
        model = Model()
        print(model.print_output())
        
    """Inventory Control"""
    def print_all_ingredients(self, threshold=5):
        f_data = self.get_item()
        inv = f_data['inventory']
        print("Ingredients inventory list (items with insufficient inventory are bolded)：")
        for item in inv['ingredients']:
            name = item['name']
            if item['stock'] <= threshold:
                name = f"\033[1m{name}\033[0m"
            print(f"- {item['ingredient_id']} {name}: {item['stock']}{item['unit']}")
        
    def add_purchase_order(self, ingredient_id, quantity):
        f_data = self.get_item()
        inv = f_data['inventory']
        ing = next((it for it in inv['ingredients'] if it['ingredient_id'] == ingredient_id), None)
        if not ing:
            print(f"Ingredient with ID {ingredient_id} not found.")
            return
        unit = ing['unit']
        price_per_unit = ing['price_per_unit']
        total_price = round(price_per_unit * quantity, 2)
        po_data = f_data['ingredient_order']
        next_id = (po_data['purchase_orders'][-1]['purchase_id'] + 1) if po_data['purchase_orders'] else 1
        po = {
            'purchase_id': next_id,
            'ingredient_id': ingredient_id,
            'ingredient_name': ing['name'],
            'quantity': f"{quantity}{unit}",
            'total_price': total_price,
            'order_time': datetime.now().isoformat(),
            'status': 'Ordered'
        }
        po_data['purchase_orders'].append(po)
        self.system.write_ingredients(self, po_data)
        print(f"ReplenishmentID {next_id}: Food {ing['name']} (ID:{ingredient_id}), Quantity {quantity}{unit}, Total price {total_price}, Time {po['order_time']}")

    def list_purchase_orders(self):
        f_data = self.get_item()
        po_dict = f_data['ingredient_order']
        if isinstance(po_dict, list):
            orders = po_dict
        else:
            orders = po_dict.get("purchase_orders", [])
        print("Current replenishment order：")
        for po in orders:
            print(f"- ReplenishmentID: {po['purchase_id']}, Food: {po['ingredient_name']} (ID:{po['ingredient_id']}), Quantity: {po['quantity']}, Total price: {po['total_price']}, Time: {po['order_time']}, Status: {po['status']}")
        return orders

    def update_purchase_order_status(self, rid, stat):
        f_data = self.get_item()
        po_dict = f_data['ingredient_order']
        orders = po_dict.get("purchase_orders", [])
        for od in orders:
            if od['purchase_id'] == int(rid):
                od['status'] =  stat
        self.system.write_ingredients(self, po_dict)
        print(f"Has updated ReplenishmentID {rid}'s status to {stat}.")
        
    def delete_purchase_order(self, purchase_id):
        f_data = self.get_item()
        po_dict = f_data['ingredient_order']
        if not isinstance(po_dict, dict) or 'purchase_orders' not in po_dict:
            print("There are no replenishment orders to delete.")
            return
        orders = po_dict['purchase_orders']
        for idx, o in enumerate(orders):
            if o['purchase_id'] == purchase_id:
                orders.pop(idx)
                self.system.write_ingredients(self, po_dict)
                print(f"Purchase order {purchase_id} has been deleted")
                return
        print(f"Purchase order {purchase_id} not found")

    """Customer Feedback"""
    def list_feedback(self):
        print("Feedbacks：")
        f_data = self.get_item()
        fk = f_data['feedbacks']
        for fb in fk.get('feedback', []):
            print(f"- ID:{fb['feedback_id']} OrderID:{fb['order_id']} CustomerID:{fb['customer_id']} DishID:{fb['dish_id']} Rating:{fb['rating']} Comment:{fb['comment']} Time:{fb['time']}")

In [65]:
def manager_interface(mgr):
    while True:
        print("\n[Manager] \nOptions: 1.Users 2.Orders 3.Inventory 4.Financial Summary 5.Feedback 6.Sign Out")
        cmd = input('> ')
        if cmd == '1': 
            print("\n[Manager] Users: \nOptions: 1.Pending approval 2.Approved 3.Quit")
            choice = input('> ')
            if choice == '1':
                mgr.panel.show_pending()
                print("\n[Manager] Users(Pending approval): \nOptions: 1.Approve 2.Reject 3.Quit")
                option = input('> ')
                if option == '1':
                    mgr.approve(input('Enter the accID to approve: '))
                elif option == '2':
                    mgr.reject(input('Enter the accID to reject: '))
                elif option == '3':
                    continue
                else:
                    print("Invalid Option")
            elif cmd == '2':
                mgr.list_approved()
                print("\n[Manager] Users(Approved): \nOptions: 1.Modify user 2.Quit")
                choice = input('> ')
                if choice == '1':
                    uid = input('Enter the accID to be modified: ')
                    nr = input('New role (optional): ')
                    np = input('New password(optional): ')
                    mgr.modify_user(uid, new_role=nr or None, new_password=np or None)
                elif choice == '2':
                    continue
                else:
                    print("Invalid Option")
            elif choice == '3':
                continue
            else:
                print("Invalid Option")
        elif cmd == '2': 
            mgr.list_pending_orders()
            print("\n[Manager] Orders: \nOptions: 1.Update order 2.Quit")
            choice = input('> ')
            if choice == '1':
                ID = int(input('OrderID: '))
                State = input('Status (Completed/In progress/Cancelled): ')
                if State == 'Completed' or State == 'In progress' or State == 'Cancelled':
                    mgr.update_order_status(ID, State)
                else:
                    print('Invalid Status')
            elif choice == '2':
                continue
            else:
                print("Invalid Option")
        elif cmd == '3': 
            mgr.print_all_ingredients()
            print("\n[Manager] Inventory: \nOptions: 1.Replenishment 3.Quit")
            choice = input('> ')
            if choice == '1':
                mgr.list_purchase_orders()
                print("\n[Manager] Inventory(Replenishment): \nOptions: 1.New Replenishment 2.Update Replenishment 3.Quit")
                option = input('> ')
                if option == '1':
                    print("Ingredient list：")
                    inv = load('datasets/ingredients.txt')
                    for it in inv['ingredients']:
                        print(f"- ID: {it['ingredient_id']} Name: {it['name']} Unit: {it['unit']} Price(RM): {it['price_per_unit']}/{it['unit']}")
                    iid = int(input('Ingredient ID: '))
                    qty = float(input('Amount (in unit): '))
                    mgr.add_purchase_order(iid, qty)
                elif option == '2':
                    pid = int(input('ReplenishmentID: '))
                    print('\n[Manager] \nOptions: 1.Update status  2.Delete')
                    state = input('> ')
                    if state == '1':
                        new = input('New status(Ordered, Received, Cancelled): ')
                        if new == 'Ordered' or new == 'Received' or new == 'Cancelled':
                            mgr.update_purchase_order_status(pid, new)
                        else:
                            print("Invalid status")
                    elif state == '2':
                        mgr.delete_purchase_order(pid)
                    elif state == '3':
                        continue
                    else:
                        print("Invalid Option")
                elif option == '3':
                    continue
                else:
                    print("Invalid Option")
            elif choice == '2':
                continue
            else:
                print("Invalid Option")
        elif cmd =='4': 
            mgr.financial_summary_past_week()
            print('\n[Manager] \nOptions: 1.AI Prediction  2.Quit')
            choice = input('> ')
            if choice == '1':
                mgr.prediction_output()
            elif choice == '2':
                continue
            else:
                print("Invalid Option")
        elif cmd =='5': 
            mgr.list_feedback()
        elif cmd == '6': 
            break
        else: print("Invalid Option")

# Cashier

In [9]:
class Cashier:
    """使用凯撒（Caesar）加解密"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted, system.caesar_shift)

    def decrypt(self, data: str, shift: int) -> str:
        result = []
        for c in data:
            if 'a' <= c <= 'z' or 'A' <= c <= 'Z':
                base = 'a' if c.islower() else 'A'
                result.append(_B_CHR(
                    (_B_ORD(c) - _B_ORD(base) - shift) % 26
                    + _B_ORD(base)
                ))
            else:
                result.append(c)
        return ''.join(result)

In [10]:
def cashier_interface():
    print("进入收银员界面...")

# Chef

In [11]:
class Chef:
    """使用简单的异或（XOR）加解密"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted, system.xor_key)

    def decrypt(self, data: str, key: int) -> str:
        chars = []
        for i in range(0, len(data), 2):
            byte = int(data[i:i+2], 16) ^ key
            chars.append(_B_CHR(byte))
        return ''.join(chars)

In [12]:
def cashier_interface():
    print("进入收银员界面...")

# Customer

In [13]:
class Customer:
    """使用乘法（Mul）加解密——每个字符的码点乘以常数"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted, system.mul_key)

    def decrypt(self, data: str, key: int) -> str:
        chars = []
        for part in data.split(','):
            num = int(part) // key
            chars.append(_B_CHR(num))
        return ''.join(chars)

In [14]:
def customer_interface():
    print("进入顾客界面...")

# Main Panel

In [15]:
class panel:
    def __init__(self):
        self.pending_users = []

    def sign_up(self, acc_name, acc_role, acc_password):
        user = {"accID": acc_name, "pass": acc_password, "role": acc_role}
        self.pending_users.append(user)
        print(f"User {acc_name}({acc_role}) has submitted a registration application and is awaiting approval by the administrator.")

    def show_pending(self):
        if not self.pending_users:
            print("There are no pending users.")
            return []
        print("List of users pending approval：")
        for u in self.pending_users:
            print(f"- {u['accID']} (Role: {u['role']})")
        return self.pending_users

In [66]:
if __name__ == '__main__':
    system = system_manager()
    pnl = panel()

    while True:
        print("\nMain Menu: 1.Sign Up  2.Login  3.Exit")
        choice = input('> ')
        if choice == '1':
            acc = input('Account Name: ')
            rol = input('Account Role(Customer/Chef/Cashier): ')
            pas = input('Account Password: ')
            if rol.lower() == 'customer':
                system.add_user({"accID": acc, "pass": pas, "role": 'customer'})
                print("Customer account created.")
            elif rol.lower() == 'chef' or rol.lower() == 'cashier':
                pnl.sign_up(acc, rol.lower(), pas)
            else:
                print("Invalid role")

        elif choice == '2':
            acc = input('Account Name: ')
            pas = input('Account Password: ')
            ok, role = system.login(acc, pas)
            if not ok:
                print("Login failed: wrong account or password.")
                continue
            print(f"Login successful, role: {role.capitalize()}")
            if role == 'manager':
                mgr = Manager(system, pnl)
                manager_interface(mgr)
            elif role.lower() == 'cashier':
                cashier_interface()
            elif role.lower() == 'chef':
                chef_interface()
            elif role.lower() == 'customer':
                customer_interface()
            else:
                print("Unknown role, unable to enter the interface.")

        elif choice == '3':
            print("Exiting the system.")
            break
            
        else:
            print("Invalid option, please try again. ")


Main Menu: 1.Sign Up  2.Login  3.Exit


>  2
Account Name:  Admin
Account Password:  Admin


Login successful, role: Manager

[Manager] 
Options: 1.Users 2.Orders 3.Inventory 4.Financial Summary 5.Feedback 6.Sign Out


>  2


List of uncompleted orders:
- OrderID: 5004, CustomerID: C104, Time: 2025-05-05T14:32:37, Status: In progress, Amount: 14.0
- OrderID: 5009, CustomerID: C110, Time: 2025-05-05T22:06:59, Status: Cancelled, Amount: 58.78
- OrderID: 5015, CustomerID: C109, Time: 2025-05-06T07:12:13, Status: Cancelled, Amount: 76.92
- OrderID: 5034, CustomerID: C101, Time: 2025-05-07T11:58:50, Status: Cancelled, Amount: 87.89
- OrderID: 5049, CustomerID: C106, Time: 2025-05-08T10:41:56, Status: Cancelled, Amount: 54.23
- OrderID: 5051, CustomerID: C106, Time: 2025-05-08T13:43:41, Status: Cancelled, Amount: 92.09
- OrderID: 5052, CustomerID: C116, Time: 2025-05-08T15:14:33, Status: Cancelled, Amount: 100.28
- OrderID: 5055, CustomerID: C113, Time: 2025-05-08T19:47:11, Status: Cancelled, Amount: 92.62
- OrderID: 5057, CustomerID: C114, Time: 2025-05-08T22:48:55, Status: Cancelled, Amount: 97.22
- OrderID: 5058, CustomerID: C114, Time: 2025-05-09T00:19:48, Status: Cancelled, Amount: 41.43
- OrderID: 5066, Cus

>  1
OrderID:  e


ValueError: invalid literal for int() with base 10: 'e'